In [119]:
import snap
import json
import pandas as pd
import statsmodels.api as sm

G = snap.LoadEdgeList(snap.PUNGraph, 'data/toronto_edgelist_knn_20.csv', 0,1, ',')
df = pd.read_csv('data/yelp_toronto.csv', sep=" ")



In [120]:
#figure out why it's ignoring the first column? literally hardcoding this shit rn
print df.columns

df['review_count'] = df['stars']
df['stars'] = df['categories']
df['categories'] = df['longitude']
df['longitude'] = df['latitude']
df['latitude'] = df['id']
df['id'] = df['idx']
df['idx'] = df.index

for i in range(len(df)):
    if not G.IsNode(i):
        G.AddNode(i)


Index([u'idx', u'id', u'latitude', u'longitude', u'categories', u'stars',
       u'review_count'],
      dtype='object')


In [121]:
#BASIC PROPERTIES TO EVALUATE: degree, clustering coefficient, vol, density
#degree
df['degree'] = df['idx'].map(lambda x: G.GetNI(int(x)).GetDeg())
#clustering coefficient
df['clust_cf'] = df['idx'].map(lambda x : snap.GetNodeClustCf(G, int(x)))

204794


In [124]:
model = sm.OLS.from_formula('review_count ~ degree + clust_cf', df).fit()
print model.summary()

                            OLS Regression Results                            
Dep. Variable:           review_count   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.004
Method:                 Least Squares   F-statistic:                     21.89
Date:                Tue, 06 Nov 2018   Prob (F-statistic):           3.28e-10
Time:                        23:43:46   Log-Likelihood:                -55182.
No. Observations:                9677   AIC:                         1.104e+05
Df Residuals:                    9674   BIC:                         1.104e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -38.5339     15.663     -2.460      0.0